# PyQCore Tutorial  
  
This tutorial is for PyQCore Library.  
In this tutorial, we use Japanese Vowels Data Set hosted by UCI.  
    https://archive.ics.uci.edu/ml/datasets/Japanese+Vowels  


In [1]:
import time
from pyqcore.client import SimpleQCoreClient
from pyqcore.examples.jpvow import load_jpvow
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

At first load data and split them for training and test data.

In [2]:
data = load_jpvow()
    # Train: 80% / Test: 20%
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data.data, data.target, test_size=0.2, random_state=1
)

Now create an API instance and get access token. Make sure that you need get license code from [QuantumCore Website](https://www.qcore.co.jp)

In [3]:
print("===QCORE(Sample)===")
# create API instance
client = SimpleQCoreClient()
# get token (login)
access_token = client.login(
         username="#USER#", password="#PASS#", endpoint="http://#ENDPOINT#"
)
print(access_token)

===QCORE(Sample)===
{'access_token':}


Now it's time for training and classification.  
Our engine adopts Rest-API interface, and SimpleQCoreClient makes it easy to convert data to json format and send it to the engine.

In [4]:
# train
start = time.time()
print(X_train.shape)
res = client.classifier_train(X=X_train, Y=y_train, access_token=access_token)
print(res)

# test
res = client.classifier_test(X=X_test, Y=y_test, access_token=access_token)
print(res)
    
# classify
res = client.classifier_predict(X=X_test, access_token=access_token)
print("acc=", accuracy_score(y_test.tolist(), res["Y"]))
print("f1=", f1_score(y_test.tolist(), res["Y"], average="weighted"))
elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time) + "[sec]")

print(res['Y'])

(512, 29, 12)
{'res': 'ok', 'train_time': 0.03760866324106852}
{'accuracy': 0.9921875, 'f1': 0.9922010869565218, 'res': 'ok'}
acc= 0.9921875
f1= 0.9922010869565218
elapsed_time:3.912313938140869[sec]
[5, 9, 7, 1, 3, 3, 4, 9, 9, 3, 8, 2, 1, 6, 9, 7, 3, 4, 6, 9, 1, 4, 8, 1, 8, 3, 7, 7, 8, 4, 8, 4, 7, 2, 6, 7, 3, 9, 4, 2, 8, 3, 7, 6, 5, 4, 2, 1, 8, 7, 2, 7, 3, 6, 5, 2, 5, 7, 1, 4, 2, 4, 8, 2, 1, 1, 8, 9, 3, 7, 4, 6, 8, 8, 3, 7, 3, 1, 6, 2, 3, 8, 7, 9, 8, 3, 7, 2, 4, 5, 3, 2, 6, 3, 5, 8, 3, 8, 6, 9, 8, 3, 6, 1, 9, 2, 3, 7, 6, 3, 4, 9, 5, 8, 8, 3, 3, 3, 1, 8, 5, 3, 9, 4, 7, 4, 1, 8]


Also, you can check the softmax values of the prediction with `softmax_top` option

In [5]:
res = client.classifier_predict(X=X_test, access_token=access_token, softmax_top=3)
res['Y'][:3]

[[[5, 0.24700839817523956],
  [6, 0.08750557154417038],
  [8, 0.08679292351007462]],
 [[9, 0.20201803743839264],
  [5, 0.09666953980922699],
  [4, 0.09265950322151184]],
 [[7, 0.2048064023256302], [1, 0.10610369592905045], [8, 0.09740772098302841]]]

Remember that our model is currently running on AWS t2.micro instace. In other words, this model runs with tiny CPU, memory and no GPU ([check out the spec here](https://aws.amazon.com/ec2/instance-types/)).  
Now let's compare to other model. In this demo, we use Logistic Regression and simple Neural Network(MLP) on Sklearn.

In [6]:
X_train = X_train.reshape(len(X_train), -1).astype(np.float64)
X_test = X_test.reshape(len(X_test), -1).astype(np.float64)
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

print("===LogisticRegression(Using Sklearn)===")
start = time.time()
lr_cls = LogisticRegression(C=9.0)
lr_cls.fit(X_train, y_train)
elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time) + "[sec]")
res = lr_cls.predict(X=X_test)
print("acc=", accuracy_score(y_test.tolist(), res))
print("f1=", f1_score(y_test.tolist(), res, average="weighted"))

print("===MLP(Using Sklearn)===")
start = time.time()
mlp_cls = MLPClassifier(hidden_layer_sizes=(100, 100, 100, 10))
mlp_cls.fit(X_train, y_train)
elapsed_time = time.time() - start
print("elapsed_time:{0}".format(elapsed_time) + "[sec]")
res = mlp_cls.predict(X=X_test)
print("acc=", accuracy_score(y_test.tolist(), res))
print("f1=", f1_score(y_test.tolist(), res, average="weighted"))

===LogisticRegression(Using Sklearn)===
elapsed_time:0.1754000186920166[sec]
acc= 0.9765625
f1= 0.9761245153216563
===MLP(Using Sklearn)===


elapsed_time:0.49468994140625[sec]
acc= 0.9453125
f1= 0.9435481250696572


Copyright (c) 2019 QuantumCore Inc.  
Github: [pyqcore](https://github.com/qcore-info/pyqcore)  
Web: [QuantumCore Inc.](https://www.qcore.co.jp)